In [1]:
!pip install --upgrade --force-reinstall transformers transformers[torch] datasets sentencepiece evaluate sacrebleu

Defaulting to user installation because normal site-packages is not writeable
  Using cached transformers-4.33.2-py3-none-any.whl (7.6 MB)
  Using cached datasets-2.14.5-py3-none-any.whl (519 kB)
  Using cached sentencepiece-0.1.99-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.3 MB)
  Using cached evaluate-0.4.0-py3-none-any.whl (81 kB)
  Using cached sacrebleu-2.3.1-py3-none-any.whl (118 kB)
  Using cached packaging-23.1-py3-none-any.whl (48 kB)
  Using cached huggingface_hub-0.17.2-py3-none-any.whl (294 kB)
  Using cached filelock-3.12.4-py3-none-any.whl (11 kB)
  Using cached requests-2.31.0-py3-none-any.whl (62 kB)
  Using cached tokenizers-0.13.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.8 MB)
  Using cached tqdm-4.66.1-py3-none-any.whl (78 kB)
  Using cached numpy-1.26.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.2 MB)
  Using cached safetensors-0.3.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.3 MB)
  Usin

In [2]:
from datasets import load_dataset

In [3]:
import pandas as pd

In [4]:
import json

In [5]:
with open('1113_tech_train_set_1195228.json') as f:
  js1 = json.loads(f.read())

df1 = pd.DataFrame(js1['data'])

In [6]:
with open('기술과학분야_한영_train_set.json') as f:
  js2 = json.loads(f.read())

df2 = pd.DataFrame(js2['data'])

In [7]:
with open('일상생활및구어체_한영_train_set.json') as f:
  js3 = json.loads(f.read())

df3 = pd.DataFrame(js3['data'])

In [8]:
koen1 = df1[['ko','en']]
koen2 = df2[['ko','en']]
koen3 = df3[['ko','en']]

ko_en_df = pd.concat([koen1,koen2,koen3])

In [9]:
ko_en_df['ko'] = ko_en_df['ko'].str.replace(pat=r'[^\w]', repl=r' ', regex=True)

In [10]:
ko_en_df['en'] = ko_en_df['en'].str.replace(pat=r'[^\w]', repl=r' ', regex=True)

In [5]:
from datasets import Dataset, DatasetDict

In [12]:
ko_en_df = ko_en_df.reset_index(drop=True)

ko_en_df

,ko,en
0,오븐 세척제에 포함된 피부 부식성 자극성 물질은 모두 심한 눈손상성 눈 자극성에도 ...,All skin corrosion irritation substances conta...
1,국내에서는 2030년 BAU 대비 37 의 온실가스 감축을 의무화함으로써 이에 따른...,In Korea it really is urgent to produce an an...
2,수술 전 정맥혈을 채취하고 월경주기의 난포기 동안 8시간 이상 금식한 후 정맥혈액을...,Before the operation blood samples were obtai...
3,3등급 이하 인정자는 8 2 만이 재가급여에서 시설급여로 변경하였다,Only 8 2 of third graders or lower changed fr...
4,센싱에 의해 공기 중 오일입자가 기준치이하인지 여부를 판단할 수 있다,It is possible to determine whether or not the...
...,...,...
3595367,그럼 미디엄은 2리터 필요하신 거죠,So then you need 2 liters of the medium right
3595368,환불 요청은 BBB 고객 지원으로 문의하십시오,Please contact BBB customer support for refund...
3595369,미니 사이즈로 언제어디서나 즐길 수 있는 휴대성을 자랑합니다,It is a mini size and boasts portability that ...
3595370,본사에서 충분히 검토하고 선정했을 것이기에 서점에서 특별한 조치는 취하지 않을 것입니다,The bookstore will not take any special action...


In [13]:
dataset = Dataset.from_pandas(ko_en_df)

dataset

Dataset({
    features: ['ko', 'en'],
    num_rows: 3595372
})

In [14]:
# dataset is already `map`'d and already has `set_format`
# 90% train, 10% test + validation
train_testvalid = dataset.train_test_split(test_size=0.1)
# Split the 10% test + valid in half test, half valid
test_valid = train_testvalid["test"].train_test_split(test_size=0.5)
# gather everyone if you want to have a single DatasetDict
datasets = DatasetDict({
    "train": train_testvalid["train"],
    "test": test_valid["test"],
    "valid": test_valid["train"]})

In [15]:
datasets

DatasetDict({
    train: Dataset({
        features: ['ko', 'en'],
        num_rows: 3235834
    })
    test: Dataset({
        features: ['ko', 'en'],
        num_rows: 179769
    })
    valid: Dataset({
        features: ['ko', 'en'],
        num_rows: 179769
    })
})

In [16]:
datasets.save_to_disk('koentalktech')

Saving the dataset (0/2 shards):   0%|          | 0/3235834 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/179769 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/179769 [00:00<?, ? examples/s]

In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [12]:
from transformers import DataCollatorForSeq2Seq

In [13]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

In [14]:
from datasets import load_dataset, load_metric

In [15]:
import numpy as np
import torch
import multiprocessing

In [16]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [17]:
model_ckpt = "KETI-AIR/ke-t5-base"
max_token_length = 64

In [18]:
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [19]:
def preprocess_function(examples):
    inputs = examples['ko']
    targets = examples['en']
    model_inputs = tokenizer(
        inputs, text_target=targets, max_length=max_token_length, truncation=True
        )
    return model_inputs

In [15]:
NUM_CPU = multiprocessing.cpu_count()
NUM_CPU

48

In [ ]:
tokenized_datasets = datasets.map(preprocess_function,
                                 batched=True,
                                 remove_columns=datasets["train"].column_names,
                                 num_proc=NUM_CPU)

In [ ]:
tokenized_datasets.save_to_disk('koentalktech_token')

In [16]:
from datasets import load_from_disk

tokenized_datasets = load_from_disk('koentalktech_token')

In [20]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

In [21]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [19]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 3235834
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 179769
    })
    valid: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 179769
    })
})

In [20]:
import evaluate

metric = evaluate.load("sacrebleu")

In [21]:
import numpy as np


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

In [30]:
training_args = Seq2SeqTrainingArguments(
    output_dir="chkpt",
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=False,
    gradient_accumulation_steps=2,
    report_to="none" # Wandb 로그 끄기
)

In [31]:
trainer = Seq2SeqTrainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [32]:
trainer.train()

trainer.save_model("./results")

Step,Training Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 24.00 MiB (GPU 0; 23.69 GiB total capacity; 21.85 GiB already allocated; 2.06 MiB free; 22.63 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [30]:
model_dir = "./chkpt/checkpoint-37920"
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)

model.cpu();

In [4]:
input_text = [
    "이번시간에는 선형회귀 알고리즘을 수학적으로 이해해보도록 하겠습니다.",
    "앞서, 살펴본 바와 같이, 선형회귀 모델은 선입니다."
]

In [6]:
inputs = tokenizer(input_text, return_tensors="pt", 
                   padding=True, max_length=64)

/home/anjinsung/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2436: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


In [7]:
inputs

{'input_ids': tensor([[  160,   669,   202,   805,   338,   290,  2919, 30044,    11,  7020,
           551,  2292,    80,   518,  1396, 12584,     3,     1],
        [  445,     4, 36513, 28902,   875,     4,   805,   338,   290,  2919,
         22276,   805,   335,     3,     1,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]])}

In [8]:
eng = model.generate(
    **inputs,
    max_length=64,
    num_beams=5,
)

eng.shape

torch.Size([2, 18])

In [9]:
[ 
    tokenizer.convert_tokens_to_string(
    tokenizer.convert_ids_to_tokens(e)) for e in eng
]

['<pad> In this time we will mathematically understand the linear regression algorithm </s>',
 '<pad> As seen above the linear regression model is a line </s><pad><pad>']

In [10]:
from datasets import load_from_disk

tokenized_datasets = load_from_disk('koentalktech_token')

In [22]:
from torch.utils.data import DataLoader

test_dataloader = DataLoader(
    tokenized_datasets["test"], batch_size=32, collate_fn=data_collator
)

In [23]:
test_dataloader_iter = iter(test_dataloader)

In [26]:
test_batch = next(test_dataloader_iter)

In [27]:
test_batch.keys()

dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'])

In [28]:
test_input = { key: test_batch[key] for key in ('input_ids', 'attention_mask') }

In [31]:
englishs = model.generate(
    **test_input,
    max_length=64,
    num_beams=5,
)

In [32]:
labels =  np.where(test_batch.labels != -100, test_batch.labels, tokenizer.pad_token_id)

In [33]:
kor_sents = tokenizer.batch_decode(test_batch.input_ids, skip_special_tokens=True)[10:20]

In [34]:
references = tokenizer.batch_decode(labels, skip_special_tokens=True)[10:20]

In [35]:
preds = tokenizer.batch_decode(englishs, skip_special_tokens=True )[10:20]

In [37]:
for s in zip(kor_sents, references, preds):
    print('Korean   :', s[0])
    print('Reference :', s[1])
    print('Translated:', s[2])
    print('\n')

Korean   : 이미 발굴 유물로 인정된 이 동전 무더기의 가치는 공식적으로 산정되지는 않았지만 100만 파운드 이상 나갈 수 있다고 이 매체는 전문가들을 인용해 전했다 
Reference : The value of this sultry coin already recognized as an excavation artifact has not been officially calculated but it could go out for more than 1 million the media reported citing experts 
Translated: The value of the plurality of coins that have already been recognized as a excavation artifact is not officially calculated but it can take more than 100 million dollars according to experts 


Korean   : 그 이유는 서로 다른n p형 반도체를 합성하게 되면 각 필름들의 다수 캐리어 농도의 영향이 생긴 것으로 판단된다 
Reference : The reason is that when different n and p type semiconductors are synthesized it is judged that the influence of the concentration of multiple carriers in each film occurs 
Translated: The reason is that when the differentn p type semiconductor is synthesized it is judged that the effect of a large number of carrier concentrations of each film has occurred 


Korean   : 저도 오늘 6시에 기상했습니다 
Reference : I als